In [ ]:
import tvm
from tvm import relay
import tvm.relay.analysis_tools

In [ ]:
program = relay.const(1) - (relay.var('x') * relay.var('y'))

In [ ]:
class GetReadableName(relay.analysis_tools.AnalysisPass):
    def visit_call(self, call):
        super().visit_call(call)
        self._add_detail(call, readable_name=call.op.name)

In [ ]:
class GetIndex(relay.analysis_tools.AnalysisPass):
    def __init__(self):
        super().__init__()
        self.__id = 0

    def visit_call(self, call):
        super().visit_call(call)
        self._add_detail(call, id=self.__id)
        self.__id += 1

In [ ]:
analyses = [GetReadableName(), GetIndex()]
analysis_results, summary_results = relay.analysis_tools.run_analyses(program, analyses)

In [ ]:
analysis_columns = relay.analysis_tools.get_analysis_columns(analysis_results)
analysis_columns

In [ ]:
records = relay.analysis_tools.get_records(analysis_results, analysis_columns)
records

In [ ]:
import pandas as pd
pd.DataFrame.from_records(records, columns=['id', 'op'], index='id')

In [ ]:
from tvm.relay.testing.mobilenet import get_workload

module, params = get_workload()

analyses = [GetReadableName(), GetIndex()]
analysis_results, summary_results = relay.analysis_tools.run_analyses(module['main'], analyses)
records = relay.analysis_tools.get_records(analysis_results, analysis_columns)
pd.DataFrame.from_records(records, columns=['id', 'op'], index='id')

In [ ]:
class SummarizeOpTypes(relay.analysis_tools.AnalysisPass):
    def _summarize(self):
        histogram = {}
        for node, data in self._existing_data.items():
            if data['readable_name'] not in histogram:
                histogram[data['readable_name']] = 1
            else:
                histogram[data['readable_name']] += 1
        self._add_summary(histogram)

In [ ]:
analyses = [GetReadableName(), GetIndex(), SummarizeOpTypes()]
analysis_results, summary_results = relay.analysis_tools.run_analyses(module['main'], analyses)
summary_results

In [ ]:
summary_columns = relay.analysis_tools.get_summary_columns(summary_results)
summary_columns

In [ ]:
from tvm.relay.testing.resnet import get_workload

module, params = get_workload()

mobilenet_summary = summary_results

analyses = [GetReadableName(), GetIndex(), SummarizeOpTypes()]
_, resnet_summary = relay.analysis_tools.run_analyses(module['main'], analyses)
resnet_columns = relay.analysis_tools.get_summary_columns(resnet_summary)

summary_columns.update(resnet_columns)
summary_columns

In [ ]:
mobilenet_record = relay.analysis_tools.summary_to_record(summary_columns, mobilenet_summary)
resnet_record = relay.analysis_tools.summary_to_record(summary_columns, resnet_summary)

pd.DataFrame.from_records([mobilenet_record, resnet_record], columns = summary_columns).fillna(0).astype('int')